In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup

import pandas as pd
import time

import uuid

import os

In [14]:
chromedriver_path = os.path.expanduser("~/GitLocal/SitShop/sitData/scraper/chromedriver_v117.exe")
s = Service(chromedriver_path)
driver = webdriver.Chrome(service = s)

In [11]:
driver = webdriver.Firefox()

In [12]:
driver.get("https://shopee.sg/Baseus-D03-Wireless-Headphones-Bluetooth-5.3-Low-Latency-With-mic-3.5mm-jack-For-Notebook-Speaker-Computer-i.131198971.17494411818?sp_atk=b6d64233-ec7c-475e-8d52-c8bbaf20822e&xptdk=b6d64233-ec7c-475e-8d52-c8bbaf20822e")

In [81]:
search_bar = driver.find_element(By.CLASS_NAME, "shopee-searchbar-input__input")
search_bar.send_keys("bottle")
search_bar.send_keys(Keys.RETURN)

In [13]:
time.sleep(2)
username_bar = driver.find_element(By.NAME, "loginKey")
username_bar.send_keys("linjericho")

time.sleep(2)
password_bar = driver.find_element(By.NAME, "password")
password_bar.send_keys("Jericho1")

time.sleep(2)
password_bar.send_keys(Keys.RETURN)

In [26]:
product_list = driver.find_elements(By.CLASS_NAME, "shopee-search-item-result__item")

for product in product_list:
    product.click()
    break

In [83]:
page_source = driver.page_source
soup = BeautifulSoup(page_source)

In [84]:
merchant = {}

merchant["name"] = soup.find("div", class_="VlDReK").text

merchant_info = soup.find("div", class_="Po6c6I")
merchant_info = merchant_info.find_all(class_="R7Q8ES")

for info in merchant_info:
    label = info.find("label").text
    span  = info.find("span").text
    merchant[label] = span

display(merchant)

{'name': 'The Corner',
 'Ratings': '11.6k',
 'response rate': '92%',
 'joined': '24 months ago',
 'products': '279',
 'response time': 'within hours',
 'follower': '14k'}

In [105]:
product = {}

product_id = str(uuid.uuid4())

product["product_id"] = product_id

product_info = soup.find("section", class_="RBf1cu")

# Name, Preferred and Mall
name_section = product_info.find("div", class_="_44qnta")
product["name"] = name_section.find("span").text

try:
    name_section.find("svg")
    product["preferred"] = 0
    product["mall"]      = 1
except:
    product["preferred"] = 0
    product["mall"]      = 0

try:
    name_section.find("div")
    product["preferred"] = 1
    product["mall"]      = 0
except:
    product["preferred"] = 0
    product["mall"]      = 0

# Ratings
rating_section = product_info.find("div", class_="X5u-5c")
product["avg_rating"]   = rating_section.find_all("div", class_="_1k47d8")[0].text
product["total_rating"] = rating_section.find_all("div", class_="_1k47d8")[-1].text
product["total_sold"]   = rating_section.find("div", class_="e9sAa2").text

# Other info
product["price"] = product_info.find("div", class_="pqTWkA").text
product["fav_count"] = soup.select_one(".flex.items-center._3jkKrB .Ne7dEf").text
product["quantity_available"] = product_info.find("section", class_="flex items-center _6lioXX").find_all("div")[-1].text

# Description
desc_section = soup.find("div", class_='f7AU53')
sentence_list = desc_section.find_all('p')
sentence_list = [sentence.text for sentence in sentence_list]
product['description'] = ' '.join(sentence_list)

display(product)

{'product_id': 'cb3975d2-46f6-4d30-a1ab-2f8aab3df188',
 'name': '1300ml Water Bottle BPA Free Leak-proof Removable Straw Free Sticker Dual Spout : Direct Spout or With Straw',
 'preferred': 1,
 'mall': 0,
 'avg_rating': '4.8',
 'total_rating': '71',
 'total_sold': '285',
 'price': '$7.90',
 'fav_count': 'Favorite (208)',
 'quantity_available': '127 pieces available',
 'description': 'Material: PC (BPA free) \n Capacity: 1300ml \n Safe Temperature: -10 to 90℃ \n Size: 17.9cm in height, 11.5 in width \n COME WITH FREE STICKER & BELT 🇸🇬🇸🇬🇸🇬 Singapore local seller, ready stock in Singapore. 🚚🚚🚚 Super fast delivery. Your parcel will be sent out within 1 working day, and arrive at your doorstep within 2-3 working days. 98% of our customers received the parcels within 2 working days since order placed. 📦📦📦 Hassle free when purchase fragile item like glass, ceramic products. Your parcels are well-protected with water-resistant packaging to minimize risk of damage during delivery. And replaceme

In [112]:
review_list = []

review_section = driver.find_elements(By.CLASS_NAME, "shopee-product-rating")

for review in review_section:
    comment =  {}
    comment["comment_id"]  = str(uuid.uuid4())
    comment["username"]    = review.find_element(By.CSS_SELECTOR, ".shopee-product-rating__author-name").text
    comment["date"]        = review.find_element(By.CSS_SELECTOR, "div[class='shopee-product-rating__time']").text.split(" | ")[0]
    comment["rating"]      = len(review.find_elements(By.CSS_SELECTOR, ".icon-rating-solid--active"))
    
    try:
        comment["comment"] = review.find_element(By.CSS_SELECTOR, "div[class='Rk6V+3']").text
    except:
        comment["comment"] = None
    
    comment["product_id"]  = product_id

{'comment_id': '89728794-5b86-493f-93cd-4b15e6560403',
 'username': 'p*****g',
 'date': '2023-06-20 21:32',
 'rating': 3,
 'comment': 'super well packaged!! large capacity. hope it can last!! :D\n\nedit: bottle leaks for me :( and the straw seems to easily detach..',
 'product_id': 'cb3975d2-46f6-4d30-a1ab-2f8aab3df188'}

{'comment_id': '742fd97d-1f4a-4a05-b709-46961f0d47b5',
 'username': 'anyawk',
 'date': '2022-11-25 23:20',
 'rating': 3,
 'comment': 'Appearance: cute\nPerformance: works\nQuality: kinda dirty when it was delivered\nSome instructions on how to assemble and use the straw would be nice… took a while to figure it out.',
 'product_id': 'cb3975d2-46f6-4d30-a1ab-2f8aab3df188'}

{'comment_id': '3f6c267b-3f88-4504-a09d-c39e1bf78c8a',
 'username': 'j*****6',
 'date': '2023-05-31 01:19',
 'rating': 3,
 'comment': None,
 'product_id': 'cb3975d2-46f6-4d30-a1ab-2f8aab3df188'}

In [118]:
review_next_page_button = driver.find_element(By.CSS_SELECTOR, ".shopee-icon-button--right")
review_next_page_button.click()

In [120]:
table = pd.DataFrame()

In [123]:
table.to_csv("../asset/some.csv")